In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
import plotly.express as px
from dotenv import load_dotenv
import json
from collections import defaultdict
import importlib
import utils
importlib.reload(utils)
import os
from scipy.stats import linregress
from sklearn.metrics import mean_absolute_error
from scipy.optimize import differential_evolution
import numpy as np
import pandas as pd
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from scipy.optimize import curve_fit
from scipy.special import beta as beta_func  # Beta function
import os
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from dotenv import load_dotenv
import json
import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open("config_MultiDataFiles.json", "r") as file:
    config = json.load(file)
model_config = utils.base_model_config()
# Load datasets for observations
def load_observations(file_path, sheet_name, cultivar_col,cultivar_name,treatment_col,treatment_name,location):
    # Define the expected column names for the final concatenated DataFrame
    TREATMENT = "Treatment"
    LOCATION = "Location"
    xls = pd.ExcelFile(file_path, engine="openpyxl")
    raw_df = pd.read_excel(xls, sheet_name = sheet_name)

    # Rename the Dates to Days of Year (DOY)
    doy_column = [pd.to_datetime(col).dayofyear for col in raw_df.columns if utils.is_date_column(col)]
    df = raw_df.rename(columns={col:pd.to_datetime(col).dayofyear for col in raw_df.columns if utils.is_date_column(col)})
    obs_year = [pd.to_datetime(col).year for col in raw_df.columns if utils.is_date_column(col)][0]


    # Apply filtering for the given cultivar & treatments
    filtered_df = df[df[cultivar_col].isin([cultivar_name]) & df[treatment_col].isin([treatment_name])]

    # Rename columns
    columns_to_select = [treatment_col] + doy_column  # Ensure it's a flat list
    filtered_df = filtered_df[columns_to_select]
    filtered_df.rename(columns={treatment_col: TREATMENT}, inplace=True)
    # Add the LOCATION column to the DataFrame
    filtered_df[LOCATION] = location

    # Remove rows with any NaN values
    filtered_df = filtered_df.dropna()

    # Reset index 
    filtered_df.reset_index(drop=True)

    PBB, BB, BudBurstDOY = utils.BB_specifications(location,filtered_df[doy_column])
    return obs_year, PBB, BB, BudBurstDOY
    


In [2]:
def objective_function(params, locations, weather_df, obs_df):
    Tc, Tmin, Topt, Tmax, chill_target, heat_target = params[:6]
    chill_start_days = params[6:]
    
    predictions = []
    for i, location in enumerate(locations):
        start_day = int(chill_start_days[i])
        for year in years_per_location[location]:
            # Subset weather for location/year
            weather = get_weather_subset(weather_df, location, year)

            # Chill accumulation
            chill = 0
            for day in weather[start_day:]:
                chill += chill_unit_function(day['temp'], Tc)
                if chill >= chill_target:
                    break

            # Heat accumulation
            heat = 0
            for day in remaining_days:
                heat += beta_heat_function(day['temp'], Tmin, Topt, Tmax)
                if heat >= heat_target:
                    predicted_doy = day['doy']
                    break

            predictions.append(predicted_doy)

    observed = get_observed_days(obs_df)
    return mean_squared_error(observed, predictions)


In [ ]:
bounds = [
    (0, 15),     # Tc
    (0, 10),     # Tmin
    (10, 30),    # Topt
    (20, 40),    # Tmax
    (20, 100),   # chill target
    (100, 1000), # heat target
] + [(200, 320)] * num_locations  # chill start days

result = differential_evolution(
    func=lambda p: objective_function(p, locations, weather_df, obs_df),
    bounds=bounds,
    strategy='best1bin',
    maxiter=1000,
    seed=42
)